# Solver Callbacks in JuMP

MIP solvers are complicated combinations of many techniques: cutting planes, heuristics, branching rules, etc.

Some solvers allow you to customize aspects of the solve process in a deeper way than just setting options for these parameters. You can provide code to be run when certain events happen, and the solver **calls back** to these functions to ask what action(s) should be taken. Why might you want to do this? Some situations:

* **Informational Callbacks:** you might want to extract the intermediate solutions found in the branch&bound tree, or other useful information during the solution process, and not work only with the optimal solution.

* **Lazy Constraints:** you might not want to enumerate all of your constraints at the onset, and enforce certain constraints only when they're needed.

We'll explore these cases in this notebook.

## Warm-Up: Linear Regression and its variants

We start with linear regression: suppose we have a data matrix $A$, and vector $b$, and we want to estimate a vector $x$ such that:

$$
\underset{x}{\min}\ || Ax-b ||_2^2
$$

One way of expressing it in JuMP is through the following formulation:

$$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z_i = a_i^\top x - b_i \quad\forall i
$$

where $a_i^\top$ is the $i$-th row of the data matrix $A$.

Let's first create the data $A$ and $b$.

In [ ]:
srand(123)

n = 20
p = 5

real_x = 10*rand(p)
A = rand(n,p)
b = A*real_x + rand(n);

Now let's formulate the problem in JuMP and solve it.

In [ ]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver())
@variable(m, x[1:p])
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])
        
solve(m)

Let's take a look at the value of $x$ we found, and compare it to its true value.

In [ ]:
@show getvalue(x);
@show real_x;

**Discussion**: When might you want to do this yourself (in JuMP/etc), versus using a commercial/opensource package?

>**\[Exercise\]**: Non-negative Least Squares

> How might we modify the formulation above to solve for non-negative least squares, i.e. $\underset{x\geq 0}{\min}\ || Ax-b ||_2^2$

>**\[Exercise\]**: Sparse Linear Regression

> How might we modify the formulation above if we know at most $k$ of the coefficients are non-zero? i.e. $\underset{x: ||x||_0\leq k}{\min}\ || Ax-b ||_2^2$

**Discussion**: Do you know of other ways of getting sparse (as in low number of non-zero coefficients) solutions in a linear regression setting?

## 1- Extracting Intermediate Solutions
Rather than having to parse the solver log, sometimes querying for information (amongst other things) might be useful for you to do convergence plots (or perform other diagnostics). Informational callbacks are added to a JuMP model with the `addinfocallback(m::Model, f::Function; when::Symbol)` function, where the `when` argument should be one of `:MIPNode`, `:MIPSol` or `:Intermediate` (listed under `cbgetstate()` in the [MathProgBase documentation](https://mathprogbasejl.readthedocs.io/en/latest/callbacks.html)).

Suppose we wish to extract all the incumbent solutions generated during the branch&bound process. Here's how we can modify the code above to do so:

In [ ]:
using JuMP, Gurobi

M = 10000
k = 2

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])

@objective(m, Min, sum(z[i]^2 for i in 1:n))

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])
@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)

### --- NEW ---

### END OF NEW

solve(m)

We can inspect the incumbent solutions generated:

In [ ]:
solutionvalues

As well as their corresponding objective values:

In [ ]:
[norm(A*sol_x-b)^2 for sol_x in solutionvalues]

**Remark**: Can you identify the objective values of the incumbent solution in the solver log above? Which of the incumbent solutions were found via heuristics, and which ones were found via branch&bound?

>**\[Exercise\]**: Information Callbacks

> How might you organize the code such that you can extract

> - the objective function value of the current solution
> - the current best bound on the optimal solution
> - the current time taken since the start of the solution process

> (Hint: http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#informational-callbacks)

## 2- Modeling using Lazy Constraints

### Motivation
**Question**: Now suppose we wish to solve the sparse linear regression problem

$$
\underset{x}{\min}\ \sum_i z_i^2 \\
\text{s.t.}\quad z_i = a_i^\top x - b \quad\forall i \\
x_j \leq My_j \\
x_j \geq -My_j \\
\sum_{j=1}^p y_j \leq k \\
y_j\in\{0,1\}\quad\forall j=1,\dots,p
$$

but as a sequence of mixed integer linear programs, rather than a single mixed integer quadratic program. How might we go about doing so?

**Answer**: ![](outerapprox.jpg)
(image source: http://www.mit.edu/~dimitrib/Polyhedral_Approx_Tsinghua.pdf)

In general, we might want to perform a "linearization" of some (often convex) function, where we perform a succession of "outer-approximations" that converges towards the underlying function we wish to optimize over.

**Remark**: For readers who are interested in cutting-plane methods, the following might be a good reference: https://web.stanford.edu/class/ee392o/localization-methods.pdf

**Discussion**: What are potential issues we might run into, if we wish to implement this?

### Description
Lazy constraints are useful when **the full set of constraints is too large to explicitly include in the initial formulation**. We might want to modify the MIP solution process such that when we have a new solution (for example with a heuristic or by solving a problem at a node in the branch-and-bound tree), we are given the opportunity to generate additional constraint(s). This allows us to generate them only upon demand, and stop the process based on our termination criteria.

There are three important steps to providing a lazy constraint callback:

1. we must write a function that will analyze the current solution that takes a single argument, e.g. `mylazycongenerator(cb)`, where cb is a reference to the callback management code inside JuMP.
2. do whatever analysis of the solution you need to inside your function to generate the new constraint before adding it to the model with `@lazyconstraint(cb, myconstraint)` (instead of the usual `@constraint(m, myconstraint)`).
3. finally we notify JuMP that this function should be used for lazy constraint generation using the `addlazycallback(m, mylazycongenerator)` function before we call `solve(m)`.

For more information, see http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#lazy-constraints

### Task
Now, we'd like to implementing sparse linear regression using lazy constraints, instead of formulating it as a quadratic optimization problem. To do so, we first represent the objective function

$$
f(z) = ||z||_2^2
$$

as the pointwise maximum of affine functions:

$$
f(z) = \underset{\beta}{\sup} ||\beta||_2^2 + 2\beta^\top(z-\beta)
$$

(or equivalently)

$$
f(z) = \min\ \eta \\
\text{s.t.}\ \ \eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall\beta
$$

### Coding it up
So our JuMP model becomes:

$$
\underset{x}{\min}\ \eta \\
\text{s.t.}\quad 
\eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall \beta \\
z_i = a_i^\top x - b \quad\forall i \\
x_j \leq My_j \\
x_j \geq -My_j \\
\sum_{j=1}^p y_j \leq k \\
y_j\in\{0,1\}\quad\forall j=1,\dots,p
$$

where we represent the set of (infinite) constraints:

$$
\eta \geq ||\beta||_2^2 + 2\beta^\top(z-\beta) \quad\forall \beta
$$

using lazy constraints.

>**\[Exercise\]**: Lazy Callbacks

> Implement the above formulation in JuMP (reference: http://www.juliaopt.org/JuMP.jl/0.15/callbacks.html#lazy-constraints)

In [ ]:
using JuMP, Gurobi

M = 10000
k = 2

m = Model(solver=GurobiSolver())
@variable(m, x[1:p] >= 0)
@variable(m, y[1:p], Bin)
@variable(m, z[1:n])
@variable(m, eta >= 0)

@objective(m, Min, eta)

@constraint(m, [i=1:n], z[i] == sum(A[i,j]*x[j] for j in 1:p) - b[i])
@constraint(m,[j=1:p], x[j] <= M*y[j])
@constraint(m,[j=1:p], x[j] >= -M*y[j])
@constraint(m, sum(y[j] for j in 1:p) <= k)

### --- Your solution here ---
function lazyleastsqs(cb)
end
addlazycallback(m, lazyleastsqs)
### --- End of solution ---

solve(m)

**Discussion**: So far, we've been looking at modelling a convex function as the pointwise maximum of a (possibly infinite) set of affine functions in the objective function. Can you anticipate other optimization problems that might be usefully modelled using a large (possibly infinite) number of linear constraints?

See for e.g. Robust Portfolio Optimization and Travelling Salesman by Iain in [last year's IAP class](https://github.com/joehuchette/OR-software-tools-2015/blob/master/7-adv-optimization/Callbacks.ipynb).

**Discussion**: Sometimes a good polyhedral outer approximation might need too many linear constraints, and might benefit from "extended formulations". See http://www.mit.edu/~mlubin/micp-cribb.pdf (from slides 19 onwards) for some details.

## 3- Application - the Traveling Salesman Problem

A canonical example of a problem where a lazy constraint formulation comes in useful is the **Traveling Salesman Problem (TSP)**.
> Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits every city exactly once?

Let's create an example where cities are random points in the 2D plane, and the distance function is just the Euclidean metric.

In [ ]:
using Gadfly, GraphPlot, LightGraphs
using JuMP, Gurobi
N = 50
srand(1)
# create random points in the plane
locations = [[rand(),rand()] for i=1:N]
locs_x = [locations[i][1] for i=eachindex(locations)]
locs_y = [locations[i][2] for i=eachindex(locations)]
# distance matrix
dists = zeros(length(locations), length(locations))
for (i, location1) in enumerate(locations), (j, location2) in enumerate(locations)
    dists[i,j] = norm(location2 - location1)
end
# show the locations
g = Graph(N)
gplot(g, locs_x, locs_y, nodelabel=collect(1:N))

### Formulation

Let's try to formulate the traveling salesman problem as an integer program.
Let the cities be indexed from 1 to N.
Let $d_{ij}$ be the distance between city $i$ and city $j$.

Decision variables: $x_{ij}=\begin{cases} 1,\quad \text{if city $i$ and city $j$ are adjacent in the shortest tour}\\
0,\quad \text{otherwise.}\end{cases}$

N.B. $x_{ij}$ and $x_{ji}$ are redundant ($x_{ij}=x_{ji}$), so we only define the variable $x_{ij}$ for $i < j$. Then we can formulate the following integer program.

$$
\underset{x}{\min}\ \sum_{i=1}^N\sum_{j=i+1}^N d_{ij}x_{ij} \\
\text{s.t.}\quad 
\sum_{j=i+1}^N x_{ij} + \sum_{j=1}^{i-1}x_{ji} = 2 \quad\forall i, 1\le i \le N \\
x_{ij}\in\{0,1\}\quad\forall i,j \text{ s.t. } 1\le i < j \le N
$$

In [ ]:
tsp = Model(solver=GurobiSolver())

# Create the binary variables

# Add the constraints that every city needs to be visited exactly once

# Define the objective to get the shortest distance

solve(tsp)

Let's take a look at the solution we found.

In [ ]:
# Helper function to plot the solution
function plotTSP(locs_x, locs_y, xVal)
    n = length(locs_x)
    g = Graph(n)
    for i=1:n, j=(i+1):n
        if xVal[i,j] > 0.5
            add_edge!(g, i, j)
        end
    end
    gplot(g, locs_x, locs_y, nodelabel=collect(1:n))
end
plotTSP(locs_x, locs_y, getvalue(x));

Yikes! Our formulation is missing something! What are some potential ways to fix it?

One common way is **subtour elimination** constraints, to prevent the final solution from having any small cycles, i.e. cycles that do not include all the nodes.

Given a subtour $S\subset \{1,\ldots,N\}$, a subtour elimination constraint looks like:
$$\sum_{i\in S} \left(\sum_{j\notin S, j > i}x_{ij}+\sum_{j\notin S, j < i}x_{ji}\right) \ge 2$$.

> **[Exercise]** How many subtours are there for $N=4$? for $N=5$? for general $N$?

As $N$ grows larger, the number of subtour elimination constraints grows exponentially. It is therefore impractical to add all of these constraints into the model.

Instead, we generate these constraints lazily. Every time Gurobi has an incumbent solution, we find the shortest subtour in the solution, and add a lazy constraint eliminating this particular subtour.

The function below finds the shortest cycle in a given solution

In [ ]:
function subtour(xVal, nNodes, criterion="shortest")
    graph = Graph(nNodes)
    for i=1:nNodes, j=(i+1):nNodes
        if xVal[i,j] > 0.5
            add_edge!(graph, i, j)
        end
    end
    visited = falses(nv(graph))
    cycles = []
    selected = [[] for i=vertices(graph)]
    for e = edges(graph)
        push!(selected[src(e)], dst(e))
        push!(selected[dst(e)], src(e))
    end
    while true
        current = findfirst(visited, false)
        thiscycle = [current]
        while true
            visited[current] = true
            neighbors = [x for x in selected[current] if !visited[x]]
            if isempty(neighbors)
                break
            end
            current = neighbors[1]
            push!(thiscycle, current)
        end
        push!(cycles, thiscycle)
        if sum(length(cycle) for cycle in cycles) == nv(graph)
            break
        end
    end
    if criterion == "shortest"
        selectedCycle = cycles[indmin([length(cycle) for cycle in cycles])]
    elseif criterion == "longest"
        selectedCycle = cycles[indmax([length(cycle) for cycle in cycles])]
    else
        selectedCycle = rand(cycles)
    end
    return selectedCycle, length(selectedCycle)
end

> **[Exercise]** Given the method `subtour` above, let's write a callback to eliminate subtours lazily.

In [ ]:
# Create model
tsp_cb = Model(solver=GurobiSolver(LazyConstraints=1))

# Define variables
@variable(tsp_cb, x[i=1:N, j=(i+1):N], Bin)

# Every city must be visited at least once
@constraint(tsp_cb, visitEveryCity[i=1:N], sum(x[i,j] for j=(i+1):N) + sum(x[j,i] for j=1:(i-1)) == 2)

# Define objective to be optimized
@objective(tsp_cb, Min, sum(dists[i,j] * x[i,j] for i=1:N, j=(i+1):N))

function subTourElimination(cb)
    cycle, length = subtour(getvalue(x), N, "shortest")
    if length < N
        # your job - insert lazy constraint here
        
    else
        # no more subtours need to be eliminated
    end
end
                
addlazycallback(tsp_cb, subTourElimination)

solve(tsp_cb)
plotTSP(locs_x, locs_y, getvalue(x))

> **[Discussion/Exercise]** Choice of subtour.

> a) At each incumbent solution, we only add a constraint removing one subtour, instead of all subtours. What do you think is the rationale behind this choice?

> b) The provided `subtour` method has a third argument, which lets you select which subtour is eliminated (options are "shortest", "longest", and "random"). Compare the effect of these three choices of subtour on the solver performance, in particular the number of incumbent solutions explored by Gurobi (HINT: how can you get this information?), and any other metrics you feel could be relevant.